In [1]:
using Laplacians

In [2]:
pwd()

"/Users/serbanstan/git/Laplacians.jl/debug/Solver Testing"

In [3]:
include("../../src/primalDualIPM.jl");

In [4]:
solvers = [augTreeSolver, KMPSDDSolver, samplingSDDSolver, AMGSolver]

4-element Array{Function,1}:
 Laplacians.augTreeSolver    
 Laplacians.KMPSDDSolver     
 Laplacians.samplingSDDSolver
 Laplacians.AMGSolver        

In [ ]:
n = 3000

curSolver = []
time = []
edges = []
iters = []
precision = []

for i in 1:100
    a = wtedChimera(n,i)
    
    Bt = -dirEdgeVertexMat(a);
    Bt = Bt';
    c1 = rand(size(Bt)[2]);
    u = 0.25*ones(size(Bt)[2]);
    b1 = 0.25*Bt*rand(size(Bt)[2]);

    m = size(Bt)[2]
    n = size(Bt)[1]
    A = [Bt spzeros(n,m); speye(m) speye(m)];
    b = [b1;u];
    
    for solver in solvers
        try
            mysolver = (H -> lapWrapSolver(solver,H,tol=1e-8,maxits=1000,maxtime=20))
            tic()
            (x1,y1,s1,numIter) = minCostFlow(Bt,b1,c1,u; lapSolver=mysolver, tol = 1e-6);

            push!(time, toc())
            push!(curSolver, solver)
            push!(edges, nnz(a))
            push!(iters, numIter)
            push!(precision, norm(A * x1 - b) / norm(b))
        catch
            print("Error for ", i, " and ", solver)
        end
    end
    
    println("Done for ", i)
end

elapsed time: 9.951754705 seconds
elapsed time: 9.553145114 seconds
elapsed time: 6.670870856 seconds
elapsed time: 21.749794858 seconds
Done for 1
elapsed time: 1.620957936 seconds
elapsed time: 1.262048678 seconds
elapsed time: 1.759752057 seconds
elapsed time: 20.164783541 seconds
Done for 2
elapsed time: 5.453728366 seconds
elapsed time: 5.062706715 seconds
elapsed time: 7.942506821 seconds
Error for 3 and Laplacians.AMGSolverDone for 3
elapsed time: 